In [1]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"
ag = "Reuters"
#reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'

In [2]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
import datetime
#from db_manager import *
#import sqlite3
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from manager import *
from threading import Lock
from functools import partial
import time
mutex = Lock()

years = ['2014',] #'2015']#, '2014', '2015', '2016', '2017']
months = ['01']#, '02', '03', '04', '05', '06']#, '07', '08', '09', '10', '11', '12']
days = ['02']#,'02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    
exec(open("db_management/DB.py").read()) 

In [3]:
add_agency(ag)

'There is already a Table with such name: Reuters'

In [4]:
def get_text(link):
    try:
        r = requests.get(link)
        #print(r.raise_for_status())
        soup = BS(r.content, 'html.parser')
        head = soup.html.title.string.strip().split(" | ")[0]
        text = '\n'.join([e.get_text() for e in soup.find_all('p')])
        #print(text)
        #print(head)
        return head, text
    
    except HTTPError:
        print("No article found by this link!", link)
            

In [8]:
def get_info(link_time, date): 
    try:
    #for link_time in links_times:
        title,text = get_text(link_time[0])
        time = link_time[1]
        print(time)
        date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')
        print(str(date))
        #print(str(date)+'\n'+title+'\n'+text+'\n'+link_time[0])
        #print(str(date)+'\n'+title+'\n'+link_time[0])
        add_news(str(date), title, text, link_time[0], ag)
    except HTTPError:
        print("No page found by this link!",r.status)
    return "Done for {}".format(str(date))

In [9]:
start = time.time()
from functools import partial
for y in years:
    for m in months:
        for d in tqdm(days):
            try:
                print(y, m, d)
                datetime.datetime(year=int(y),month=int(m),day=int(d))
                r = requests.get(link.format(y, m, d), timeout=5)
                if r.ok: # check if page found
                    soup = BS(r.content, 'html.parser')
                    span_list = soup.find_all('div' , attrs={'class':"headlineMed"})
                    header = str(soup.find_all('title')).split()
                    day, month, year = header[6], header[7], header[8]

                    links_times = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list 
                                   if not str(el).split('>')[1].split("=")[1].replace('"', '').startswith('http://www.reuters.com/news/video')]
                    

                    infer = partial(get_info, date=[day, month, year])
                    
                    pool = Pool(processes=cpu_count())
                    
                    pool.map(infer, links_times)
                    pool.close()
                    pool.join()
                    print(start - time.time())
                    #except:
                        #print("Terminating the pool")
                        #pool.terminate() 
            except HTTPError:
                print("No page found by this link!",r.status)
                continue
            except ValueError:
                print("Day doesn't exist")
                continue
                    
print("DONE")

  0%|          | 0/1 [00:00<?, ?it/s]

2014 01 02
4:01PM
4:35PM
6:26PM
2014-01-02 16:01:00
2014-01-02 18:26:00
2014-01-02 16:35:00


11:57PM


OperationalError: (psycopg2.OperationalError) FATAL:  remaining connection slots are reserved for non-replication superuser connections
 (Background on this error at: http://sqlalche.me/e/e3q8)

2014-01-02 23:57:00
1:06PM
12:05PM
2014-01-02 13:06:00
2:43PM
2014-01-02 12:05:00
2014-01-02 14:43:00
11:14AM
2014-01-02 11:14:00
10:11AM
2014-01-02 10:11:00
8:13AM
2014-01-02 08:13:00
8:50AM
9:21AM
2014-01-02 08:50:00
2014-01-02 09:21:00


Process ForkPoolWorker-9:
Traceback (most recent call last):
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
  File "/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
Process ForkPoolWorker-12:
Traceback (most recent call last):
TypeError: getresponse() got an unexpected keyword argument 'buffering'
Traceback (most recent call last):

During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/mult

  File "/usr/lib/python3.5/http/client.py", line 1197, in getresponse
    response.begin()
  File "/usr/lib/python3.5/http/client.py", line 1151, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 1102, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.5/http/client.py", line 934, in _send_output
    self.send(msg)
  File "/usr/lib/python3.5/http/client.py", line 488, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.5/http/client.py", line 877, in send
    self.connect()
  Fi

In [ ]:
try:
    r = requests.get('https://www.reuters.com/resources/archive/us/20140111.html')
    r.raise_for_status()
except HTTPError:
    print("No page found by this link! "+ str(r.status_code))

In [ ]:
r.ok

In [ ]:
d = ['01', 'May', '2014']
t = '11:59PM'
date = datetime.datetime.strptime("{} {} {} {}".format(d[1], d[0], d[-1], t), '%b %d %Y %I:%M%p')

In [ ]:
date

In [ ]:
def d(d):
    print(d*2)
    return d
pool = Pool(processes=100)
pool.map(d, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]*100)
pool.close()
pool.join()

4
